In [1]:
# Libraries for data preparation & visualization
import numpy as np
import pandas as pd

# Ignore printing warnings for general readability
import warnings 
warnings.filterwarnings("ignore")

# pip install scikit-surprise
# Importing libraries for model building & evaluation
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import accuracy
import random

In [2]:
# Loading the dataset 
def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=';',error_bad_lines=False,warn_bad_lines=False,encoding='latin-1')
    return df

book   = loaddata("./dataset/BX-Books")
user   = loaddata("./dataset/BX-Users")
rating = loaddata("./dataset/BX-Book-Ratings")

In [3]:
rating_users = rating['User-ID'].value_counts().reset_index().\
               rename({'Index':'User-ID','User-ID':'Rating'}, axis=1)
rating_books = rating['ISBN'].value_counts().reset_index().\
               rename({'Index':'ISBN','ISBN':'Rating'}, axis=1)
# In order to avoid rating bias & for making good recommendations, limit the dataset to only those
# users that have made at least 250 ratings & books that have received at least 50 ratings

rating = rating[rating['User-ID'].isin(rating_users[rating_users['Rating']>=250]['index'])]
rating = rating[rating['ISBN'].isin(rating_books[rating_books['Rating']>=50]['index'])]

rating

,User-ID,ISBN,Book-Rating
1456,277427,002542730X,10
1468,277427,006092988X,0
1469,277427,0060930535,0
1470,277427,0060932139,0
1471,277427,0060934417,0
...,...,...,...
1147440,275970,1400031354,0
1147441,275970,1400031362,0
1147470,275970,1558744606,0
1147517,275970,1573229725,0


In [4]:
# For the recommendation system, it is prefered to have the book titles rather than ISBN for easier interpretation

rating = rating.merge(book, on="ISBN")[['User-ID','ISBN','Book-Rating','Book-Title']] # merging with the book dataframe
rating    

,User-ID,ISBN,Book-Rating,Book-Title
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...
...,...,...,...,...
80889,234828,0345333926,8,Ringworld
80890,236283,0345333926,0,Ringworld
80891,249628,0345333926,0,Ringworld
80892,261829,0345333926,0,Ringworld


# Using surprise for data with zeros

In [5]:
# creating a surprise object

reader = Reader(rating_scale=(0, 10))
# data_nonzero   = Dataset.load_from_df(ratings_explicit[['User-ID','ISBN','Book-Rating']], reader)
data  = Dataset.load_from_df(rating[['User-ID','ISBN','Book-Rating']], reader)


# Split the data into training & testing sets. Python's surprise documentation has the steps detailed out
# https://surprise.readthedocs.io/en/stable/FAQ.html

raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)                 # shuffle dataset

threshold   = int(len(raw_ratings)*0.8)

train_raw_ratings = raw_ratings[:threshold] # 80% of data is trainset
test_raw_ratings  = raw_ratings[threshold:] # 20% of data is testset

data.raw_ratings = train_raw_ratings        # data is now the trainset
trainset         = data.build_full_trainset() 
testset          = data.construct_testset(test_raw_ratings)


In [6]:
# Trying KNN (K-Nearest Neighbors) & SVD (Singluar Value decomposition) algorithms using default model parameters

models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

for model in models:
    # perform 5 fold cross validation
    # evaluation metrics: mean absolute error & root mean square error
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    # storing the average score across the 5 fold cross validation for each model
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

In [7]:
performance_df = pd.DataFrame.from_dict(results)
print("Model Performance: \n")
performance_df.T.sort_values(by='RMSE')

Model Performance: 



,MAE,RMSE,fit_time,test_time
knns.KNNWithMeans,2.337936,3.290046,0.087621,0.480908
knns.KNNBaseline,2.345825,3.296260,0.248057,0.620688
knns.KNNWithZScore,2.317795,3.323113,0.116425,0.510515
knns.KNNBasic,2.434953,3.502815,0.138630,0.472709


In [8]:
ratings_explicit=rating[rating['Book-Rating']!=0]
ratings_implicit=rating[rating['Book-Rating']==0]
print(ratings_explicit.shape)
print(ratings_implicit.shape)

(18651, 4)
(62243, 4)


In [9]:
# Hyperparameter tuning - KNNWithMeans

param_grid = { 'sim_options' : {'name': ['msd','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNNWithMeans.fit(data)

print(f'MAE Best Parameters:  {gridsearchKNNWithMeans.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNWithMeans.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNWithMeans.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNWithMeans.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson', 'min_support': 1, 'user_based': False}}
MAE Best Score:       2.3172715534003

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
RMSE Best Score:      3.220516010252885



In [10]:
# Hyperparameter tuning - KNNBasic

param_grid = { 'sim_options' : {'name': ['msd','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNBasic = GridSearchCV(KNNBasic, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNNBasic.fit(data)

print(f'MAE Best Parameters:  {gridsearchKNNBasic.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNBasic.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNBasic.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNBasic.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
MAE Best Score:       2.2855527244973173

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
RMSE Best Score:      3.202057931853683



In [11]:
# Hyperparameter tuning - KNNWithZScore

param_grid = { 'sim_options' : {'name': ['msd','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNN = GridSearchCV(KNNWithZScore, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNN.fit(data)

print(f'MAE Best Parameters:  {gridsearchKNN.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNN.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNN.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNN.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson', 'min_support': 1, 'user_based': False}}
MAE Best Score:       2.29756785395199

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
RMSE Best Score:      3.246652067117833



In [12]:
# Hyperparameter tuning - KNNBaseLine

param_grid = { 'sim_options' : {'name': ['msd','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNN = GridSearchCV(KNNBaseline, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNN.fit(data)

print(f'MAE Best Parameters:  {gridsearchKNN.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNN.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNN.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNN.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
MAE Best Score:       2.2963771885121864

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
RMSE Best Score:      3.1901441065921707



In [13]:
sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}

algo = KNNBaseline(sim_options=sim_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.1719  3.2023  3.1935  3.2216  3.1762  3.1931  0.0181  
MAE (testset)     2.2910  2.3066  2.3065  2.3140  2.2939  2.3024  0.0086  
Fit time          1.27    1.34    1.31    1.23    1.31    1.29    0.04    
Test time         1.47    1.53    1.35    1.33    1.4

In [14]:
sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}

algo = KNNWithZScore(sim_options=sim_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.2549  3.2123  3.2350  3.2500  3.2588  3.2422  0.0170  
MAE (testset)     2.3286  2.2973  2.3233  2.3234  2.3322  2.3209  0.0123  
Fit time          1.46    1.78    1.82    1.98    1.83    1.77    0.17    
Test time         1.84    1.63    1.68    1.76    1

In [15]:
sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}

algo = KNNWithMeans(sim_options=sim_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.2651  3.2102  3.1931  3.2185  3.2197  3.2213  0.0239  
MAE (testset)     2.3686  2.3308  2.3220  2.3572  2.3363  2.3430  0.0173  
Fit time          1.69    1.68    1.63    1.61    1.61    1.64    0.03    
Test time         1.48    1.68    1.45    1.44    1.

In [16]:
sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}

algo = KNNBasic(sim_options=sim_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.2299  3.1910  3.2212  3.2000  3.1492  3.1983  0.0282  
MAE (testset)     2.3045  2.2833  2.3020  2.2824  2.2391  2.2823  0.0235  
Fit time          1.73    1.55    1.71    1.37    1.61    1.60    0.13    
Test time         1.52    1.53    1.29    1.43    1.37  

In [17]:
# Model fit & prediction - KNNBasic

sim_options = {'name':'pearson_baseline','min_support':1,'user_based':False}
final_model = KNNBasic(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  2.2862
RMSE: 3.1626
MAE: 2.286163754572759, RMSE: 3.1625709633733416


In [18]:
# Model fit & prediction - KNNWithMeans

sim_options = {'name':'pearson_baseline','min_support':1,'user_based':False}
final_model = KNNWithMeans(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  2.3411
RMSE: 3.1877
MAE: 2.341146623198473, RMSE: 3.1877197422658297


In [19]:
# Model fit & prediction - KNNBaseline

sim_options = {'name':'pearson_baseline','min_support':1,'user_based':False}
final_model = KNNBaseline(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  2.3015
RMSE: 3.1533
MAE: 2.301498061070473, RMSE: 3.153308159699425


In [20]:
# Model fit & prediction - KNNWithZScore

sim_options = {'name':'pearson_baseline','min_support':1,'user_based':False}
final_model = KNNWithZScore(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  2.3259
RMSE: 3.2097
MAE: 2.32593753390686, RMSE: 3.209677185523047


In [21]:
# KNNBasic

def generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend =10):
    
    ''' This function generates "get_recommend" number of book recommendations using 
        KNNWithMeans & item based filtering. The function needs as input three 
        different parameters:
        (1) userID i.e., userID for which recommendations need to be generated 
        (2) like_recommend i.e., number of top recommendations for the userID to be 
        considered for making recommendations 
        (3) get_recommend i.e., number of recommendations to generate for the userID
        Default values are: userID=13552, like_recommend=5, get_recommend=10
    '''
    
    # Compute item based similarity matrix
    sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}
    similarity_matrix = KNNBasic(sim_options=sim_options).fit(trainset).\
                        compute_similarities() 
    
    userID      = trainset.to_inner_uid(userID)    # converts the raw userID to innerID
    userRatings = trainset.ur[userID]              # method .ur takes user innerID & 
                                                   # returns back user ratings
    
    
    # userRatings is a list of tuples [(,),(,),(,)..]. Each tuple contains item & rating
    # given by the user for that item. Next, the tuples will be sorted within the list 
    # in decreasing order of rating. Then top 'like_recommend' items & ratings are extracted
    
    temp_df = pd.DataFrame(userRatings).sort_values(by=1, ascending=False).\
              head(like_recommend)
    userRatings = temp_df.to_records(index=False) 
    
    # for each (item,rating) in top like_recommend user items, multiply the user rating for
    # the item with the similarity score (later is obtained from item similarity_matrix) for
    # all items. This helps calculate the weighted rating for all items. The weighted ratings 
    # are added & divided by sum of weights to estimate rating the user would give an item
    
    recommendations   = {}

    for user_top_item, user_top_item_rating  in userRatings:

        all_item_indices          =   list(pd.DataFrame(similarity_matrix)[user_top_item].index)
        all_item_weighted_rating  =   list(pd.DataFrame(similarity_matrix)[user_top_item].values*\
                                          user_top_item_rating)
        
        all_item_weights          =   list(pd.DataFrame(similarity_matrix)[user_top_item].values)
        
        
        # All items & final estimated ratings are added to a dictionary called recommendations
        
        for index in range(len(all_item_indices)):
            if index in recommendations:
                # sum of weighted ratings
                recommendations[index] += all_item_weighted_rating[index]        
            else:                        
                recommendations[index]  = all_item_weighted_rating[index]

    
    for index in range(len(all_item_indices)):                               
            if all_item_weights[index]  !=0:
                # final ratings (sum of weighted ratings/sum of weights)
                recommendations[index]   =recommendations[index]/\
                                          (all_item_weights[index]*like_recommend)
                      

    # convert dictionary recommendations to a be a list of tuples [(,),(,),(,)]
    # with each tuple being an item & estimated rating user would give that item
    # sort the tuples within the list to be in decreasing order of estimated ratings

    temp_df = pd.Series(recommendations).reset_index().sort_values(by=0, ascending=False)
    recommendations = list(temp_df.to_records(index=False))
    
    # return get_recommend number of recommedations (only return items the user 
    # has not previously rated)
    
    final_recommendations = []
    count = 0
    
    for item, score in recommendations:
        flag = True
        for userItem, userRating in trainset.ur[userID]:
            if item == userItem: 
                flag = False       # If item in recommendations has not been rated by user, 
                break              # add to final_recommendations
        if flag == True:
            final_recommendations.append(trainset.to_raw_iid(item)) 
            count +=1              # trainset has the items stored as inner id,  
                                   # convert to raw id & append 
            
        if count > get_recommend:  # Only get 'get_recommend' number of recommendations
            break
    
    return(final_recommendations)

In [22]:
recommendationsKNN = generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend=10)
recommendationsKNN

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


['0891073906',
 '0441003257',
 '0345409973',
 '0385498721',
 '0671793535',
 '0441790348',
 '0061099325',
 '0553288342',
 '0312978383',
 '0553580221',
 '044922046X']

In [23]:
red = pd.DataFrame(recommendationsKNN,columns = ['ISBN'])
red_ = red.merge(book, on="ISBN")[['ISBN','Book-Title']]
red_

,ISBN,Book-Title
0,0891073906,This Present Darkness
1,0441003257,Good Omens
2,0345409973,The Cobra Event
3,0385498721,Survivor : A Novel
4,0671793535,A Rose For Her Grave &amp; Other True Cases (A...
5,0441790348,Stranger in a Strange Land (Remembering Tomorrow)
6,0061099325,Coyote Waits (Joe Leaphorn/Jim Chee Novels)
7,0553288342,Sleepwalk
8,0312978383,Winter Solstice
9,0553580221,False Memory


In [24]:
# KNNBaseline

def generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend =10):
    
    ''' This function generates "get_recommend" number of book recommendations using 
        KNNWithMeans & item based filtering. The function needs as input three 
        different parameters:
        (1) userID i.e., userID for which recommendations need to be generated 
        (2) like_recommend i.e., number of top recommendations for the userID to be 
        considered for making recommendations 
        (3) get_recommend i.e., number of recommendations to generate for the userID
        Default values are: userID=13552, like_recommend=5, get_recommend=10
    '''
    
    # Compute item based similarity matrix
    sim_options       = {'name':'pearson_baseline','min_support':5,'user_based':False}
    similarity_matrix = KNNBaseline(sim_options=sim_options).fit(trainset).\
                        compute_similarities() 
    
    userID      = trainset.to_inner_uid(userID)    # converts the raw userID to innerID
    userRatings = trainset.ur[userID]              # method .ur takes user innerID & 
                                                   # returns back user ratings
    
    
    # userRatings is a list of tuples [(,),(,),(,)..]. Each tuple contains item & rating
    # given by the user for that item. Next, the tuples will be sorted within the list 
    # in decreasing order of rating. Then top 'like_recommend' items & ratings are extracted
    
    temp_df = pd.DataFrame(userRatings).sort_values(by=1, ascending=False).\
              head(like_recommend)
    userRatings = temp_df.to_records(index=False) 
    
    # for each (item,rating) in top like_recommend user items, multiply the user rating for
    # the item with the similarity score (later is obtained from item similarity_matrix) for
    # all items. This helps calculate the weighted rating for all items. The weighted ratings 
    # are added & divided by sum of weights to estimate rating the user would give an item
    
    recommendations   = {}

    for user_top_item, user_top_item_rating  in userRatings:

        all_item_indices          =   list(pd.DataFrame(similarity_matrix)[user_top_item].index)
        all_item_weighted_rating  =   list(pd.DataFrame(similarity_matrix)[user_top_item].values*\
                                          user_top_item_rating)
        
        all_item_weights          =   list(pd.DataFrame(similarity_matrix)[user_top_item].values)
        
        
        # All items & final estimated ratings are added to a dictionary called recommendations
        
        for index in range(len(all_item_indices)):
            if index in recommendations:
                # sum of weighted ratings
                recommendations[index] += all_item_weighted_rating[index]        
            else:                        
                recommendations[index]  = all_item_weighted_rating[index]

    
    for index in range(len(all_item_indices)):                               
            if all_item_weights[index]  !=0:
                # final ratings (sum of weighted ratings/sum of weights)
                recommendations[index]   =recommendations[index]/\
                                          (all_item_weights[index]*like_recommend)
                      

    # convert dictionary recommendations to a be a list of tuples [(,),(,),(,)]
    # with each tuple being an item & estimated rating user would give that item
    # sort the tuples within the list to be in decreasing order of estimated ratings

    temp_df = pd.Series(recommendations).reset_index().sort_values(by=0, ascending=False)
    recommendations = list(temp_df.to_records(index=False))
    
    # return get_recommend number of recommedations (only return items the user 
    # has not previously rated)
    
    final_recommendations = []
    count = 0
    
    for item, score in recommendations:
        flag = True
        for userItem, userRating in trainset.ur[userID]:
            if item == userItem: 
                flag = False       # If item in recommendations has not been rated by user, 
                break              # add to final_recommendations
        if flag == True:
            final_recommendations.append(trainset.to_raw_iid(item)) 
            count +=1              # trainset has the items stored as inner id,  
                                   # convert to raw id & append 
            
        if count > get_recommend:  # Only get 'get_recommend' number of recommendations
            break
    
    return(final_recommendations)

In [25]:
recommendationsKNN = generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend=10)
recommendationsKNN

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


['0553580221',
 '0440224624',
 '0446604801',
 '0060959037',
 '0553583980',
 '0441790348',
 '0449006522',
 '0553288342',
 '0515132020',
 '0446350109',
 '0446604402']

In [26]:
red = pd.DataFrame(recommendationsKNN,columns = ['ISBN'])
red_ = red.merge(book, on="ISBN")[['ISBN','Book-Title']]
red_

,ISBN,Book-Title
0,0553580221,False Memory
1,0440224624,The Loop
2,0446604801,Jack &amp; Jill (Alex Cross Novels)
3,0060959037,Prodigal Summer: A Novel
4,0553583980,True Blue
5,0441790348,Stranger in a Strange Land (Remembering Tomorrow)
6,0449006522,Manhattan Hunt Club
7,0553288342,Sleepwalk
8,0515132020,Heaven and Earth (Three Sisters Island Trilogy)
9,0446350109,Windmills of the Gods


In [27]:
# KNNWithMeans

def generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend =10):
    
    ''' This function generates "get_recommend" number of book recommendations using 
        KNNWithMeans & item based filtering. The function needs as input three 
        different parameters:
        (1) userID i.e., userID for which recommendations need to be generated 
        (2) like_recommend i.e., number of top recommendations for the userID to be 
        considered for making recommendations 
        (3) get_recommend i.e., number of recommendations to generate for the userID
        Default values are: userID=13552, like_recommend=5, get_recommend=10
    '''
    
    # Compute item based similarity matrix
    sim_options       = {'name':'pearson_baseline','min_support':1,'user_based':False}
    similarity_matrix = KNNWithMeans(sim_options=sim_options).fit(trainset).\
                        compute_similarities() 
    
    userID      = trainset.to_inner_uid(userID)    # converts the raw userID to innerID
    userRatings = trainset.ur[userID]              # method .ur takes user innerID & 
                                                   # returns back user ratings
    
    
    # userRatings is a list of tuples [(,),(,),(,)..]. Each tuple contains item & rating
    # given by the user for that item. Next, the tuples will be sorted within the list 
    # in decreasing order of rating. Then top 'like_recommend' items & ratings are extracted
    
    temp_df = pd.DataFrame(userRatings).sort_values(by=1, ascending=False).\
              head(like_recommend)
    userRatings = temp_df.to_records(index=False) 
    
    # for each (item,rating) in top like_recommend user items, multiply the user rating for
    # the item with the similarity score (later is obtained from item similarity_matrix) for
    # all items. This helps calculate the weighted rating for all items. The weighted ratings 
    # are added & divided by sum of weights to estimate rating the user would give an item
    
    recommendations   = {}

    for user_top_item, user_top_item_rating  in userRatings:

        all_item_indices          =   list(pd.DataFrame(similarity_matrix)[user_top_item].index)
        all_item_weighted_rating  =   list(pd.DataFrame(similarity_matrix)[user_top_item].values*\
                                          user_top_item_rating)
        
        all_item_weights          =   list(pd.DataFrame(similarity_matrix)[user_top_item].values)
        
        
        # All items & final estimated ratings are added to a dictionary called recommendations
        
        for index in range(len(all_item_indices)):
            if index in recommendations:
                # sum of weighted ratings
                recommendations[index] += all_item_weighted_rating[index]        
            else:                        
                recommendations[index]  = all_item_weighted_rating[index]

    
    for index in range(len(all_item_indices)):                               
            if all_item_weights[index]  !=0:
                # final ratings (sum of weighted ratings/sum of weights)
                recommendations[index]   =recommendations[index]/\
                                          (all_item_weights[index]*like_recommend)
                      

    # convert dictionary recommendations to a be a list of tuples [(,),(,),(,)]
    # with each tuple being an item & estimated rating user would give that item
    # sort the tuples within the list to be in decreasing order of estimated ratings

    temp_df = pd.Series(recommendations).reset_index().sort_values(by=0, ascending=False)
    recommendations = list(temp_df.to_records(index=False))
    
    # return get_recommend number of recommedations (only return items the user 
    # has not previously rated)
    
    final_recommendations = []
    count = 0
    
    for item, score in recommendations:
        flag = True
        for userItem, userRating in trainset.ur[userID]:
            if item == userItem: 
                flag = False       # If item in recommendations has not been rated by user, 
                break              # add to final_recommendations
        if flag == True:
            final_recommendations.append(trainset.to_raw_iid(item)) 
            count +=1              # trainset has the items stored as inner id,  
                                   # convert to raw id & append 
            
        if count > get_recommend:  # Only get 'get_recommend' number of recommendations
            break
    
    return(final_recommendations)

In [28]:
recommendationsKNN = generate_recommendationsKNN(userID=13552, like_recommend=40, get_recommend=10)
recommendationsKNN

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


['0891073906',
 '0441003257',
 '0345409973',
 '0385498721',
 '0671793535',
 '0441790348',
 '0061099325',
 '0553288342',
 '0312978383',
 '0553580221',
 '044922046X']

In [29]:
red = pd.DataFrame(recommendationsKNN,columns = ['ISBN'])
red_ = red.merge(book, on="ISBN")[['ISBN','Book-Title']]
red_

,ISBN,Book-Title
0,0891073906,This Present Darkness
1,0441003257,Good Omens
2,0345409973,The Cobra Event
3,0385498721,Survivor : A Novel
4,0671793535,A Rose For Her Grave &amp; Other True Cases (A...
5,0441790348,Stranger in a Strange Land (Remembering Tomorrow)
6,0061099325,Coyote Waits (Joe Leaphorn/Jim Chee Novels)
7,0553288342,Sleepwalk
8,0312978383,Winter Solstice
9,0553580221,False Memory


# Using sklearn NearestNeighbor

In [30]:
used_rating = ratings_explicit

ratings_matrix = used_rating.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating')
print(ratings_matrix.shape)
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix

(662, 2144)


ISBN,000649840X,0007110928,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277427,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Importing libraries for model building & evaluation 
from sklearn.neighbors import NearestNeighbors
import sklearn
import re

In [40]:
# KNN Function
#setting default variables
base_k = 40
base_user_id = 13552

default_k = 40
default_metric='cosine'
default_algorithm = 'brute'

print(default_k)

40


In [41]:
def findksimilarusers(user_id, rate_matrix, metric=default_metric, algo=default_algorithm, k=default_k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = algo)
    model_knn.fit(rate_matrix.values)
    
    loc = rate_matrix.index.get_loc(user_id)
    # Return k nearest neighbors' indices and distances to the user_id
    distances, indices = model_knn.kneighbors(rate_matrix.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    
    # Get similarity level by (1 - differences)
    similarities = 1 - distances.flatten()
    indices = indices.flatten()
    
    return similarities, indices

In [42]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, rate_matrix, similarities, indices):
    prediction = 0
    similarity_sum = np.sum(similarities) - 1 # Minus the current user (similarity = 1)
    weighted_sum = 0
    product = 1

    item_loc = rate_matrix.columns.get_loc(item_id)
    user_loc = rate_matrix.index.get_loc(user_id)
    mean_rating = rate_matrix.iloc[user_loc, :].mean()

    # For each similar user:
    for i in range(0, len(indices)):
        if indices[i] == user_loc:
            continue;
        else:
            # Get the rating differences from the mean (of other items), multiply by similarities with the selected user
            ratings_diff = rate_matrix.iloc[indices[i], item_loc] - np.mean(rate_matrix.iloc[indices[i],:])
            product = ratings_diff * (similarities[i])
            weighted_sum = weighted_sum + product
    
    # weighted_sum = summation< (x - mean) * sim > / sumsim
                                                     
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >10:
        prediction = 10
    
    # Calculate rating by weighted mean of ratings and similarity score of k similar users
    prediction = int(round(mean_rating + (weighted_sum/similarity_sum)))
    # print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [43]:
def recommendedItem(user_id, rate_matrix, metric=default_metric, algorithm=default_algorithm, k=default_k):
    if (user_id not in rate_matrix.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list : \n\n {}".format(re.sub('[\[\]]', '', np.array_str(rate_matrix.index.values))))
    else:
        prediction = []
        # Similar users based on cosine similarity
        similarities, indices = findksimilarusers(user_id, rate_matrix, metric, algorithm, k)
        
        total_book = rate_matrix.shape[1]
        for i in range(total_book):
            item_id = str(rate_matrix.columns[i])
            if (rate_matrix[item_id][user_id] != 0): # Not rated already
                print("\r{0:<0.2f}% Finished".format(i*100/total_book), end='')
                prediction.append(predict_userbased(user_id, item_id, rate_matrix, similarities, indices))
            else:                    
                prediction.append(-1) # For books that user already rated

        print("\r100.00% Finished")
        print("...")
        
        prediction = pd.Series(prediction)
        prediction = prediction.sort_values(ascending = False)
        
        # rmse_score = rmse_on_test_set(rate_matrix, similarities, indices)
        # print("RMSE Score: ", rmse_score)
        
        recommended = prediction[:10]
        print("\nk = {0}".format(k))
        print("For the User-based ({0} - {1}) approach, the following books are recommended to user {2}: \n".\
              format(algorithm, metric, user_id))
        for i in range(len(recommended)):
            print("{0}. Rated {1}: {2}".format(i+1, prediction.values[i], book['Book-Title'][recommended.index[i]]))

In [44]:
metric = 'cosine'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - cosine) approach, the following books are recommended to user 13552: 

1. Rated 3: The No. 1 Ladies' Detective Agency (Today Show Book Club #8)
2. Rated 3: Hiding Place
3. Rated 2: The Best of Larry King Live: The Greatest Interviews
4. Rated 2: The Little Book of Chocolate (Little Book Of...(Flammarion))
5. Rated 2: Paroles de Poilus : Lettres et carnets du front 1914-1918
6. Rated 2: The Silver Chair (full color) (Narnia)
7. Rated 2: Circles in the Stream (Avalon: Web of Magic, 1)
8. Rated 2: The Janitor's Boy
9. Rated 2: Rain Uk
10. Rated 2: College Majors and Careers: A Resource Guide for Effective Life Planning (College Majors and Careers)


In [45]:
metric = 'euclidean'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - euclidean) approach, the following books are recommended to user 13552: 

1. Rated 1: The Best of Larry King Live: The Greatest Interviews
2. Rated 1: La dona dels ulls de pluja: Barcelona anys 90 (El BalancÃ­)
3. Rated 1: The Door to December
4. Rated 1: College Majors and Careers: A Resource Guide for Effective Life Planning (College Majors and Careers)
5. Rated 1: Smilla's Sense of Snow
6. Rated 1: A Little Princess
7. Rated 1: Falling Leaves: The True Story of an Unwanted Chinese Daughter
8. Rated 1: The Art of Travel
9. Rated 1: Ender's Game (Ender Wiggins Saga (Paperback))
10. Rated 1: Hiding Place


In [46]:
metric = 'correlation'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - correlation) approach, the following books are recommended to user 13552: 

1. Rated 3: The No. 1 Ladies' Detective Agency (Today Show Book Club #8)
2. Rated 3: Hiding Place
3. Rated 2: West of Dodge: Frontier Stories
4. Rated 2: The New Oxford Book of Light Verse (Oxford paperbacks)
5. Rated 2: A New Germany in a New Europe
6. Rated 2: Teen Angst? Naaah . . . A Quasi-autobiography
7. Rated 2: Sofies Welt
8. Rated 2: Chasing the Dime
9. Rated 2: Driving Mr. Albert: A Trip Across America With Einstein's Brain
10. Rated 2: The Door to December
